# Esercizio 3 - SemEval

### Consegna

#### Parte 1

1. **Annotare** il file con i valori di similarità dati da noi e con i `BabelNetSynsetID` scelti da noi dal file `SemEval17_IT_senses2synsets.txt`.
   
2. **Agreement:** Dopo di che si calcola il `valore medio` di similarità, `Pearson` e `Spearman`
   
3. **Ricerca Automatica del Synset:** Andiamo a cercare il `BabelNetSynsetID` nel file dato dal prof `mini_NASARI.tsv`
   andando a massimizzare la *cosine similarity*. I valori che massimizzeranno quest'ultima saranno i nostri concetti.

   Per far ciò andiamo ad estrarre tutti i synsets relativi ad un termine, dal file `SemEval17_IT_senses2synsets.txt`
   e andiamo a calcolare la *cosine similarity* per ogni synset (quelli dei due termini) presente nel file `mini_NASARI.tsv`
   e salviamo i due sysnets che massimizzano la *cosine similarity*.

   Facendo ciò otteniamo due synset, relativi ai termini, e un valore di similarità calcolato dal file NASARI.

4. **Valutazione Similarità:** Sempre con `Pearson` e `Spearman`, tra i nostri valori di similarità e quelli trovati
   al passo precedente.

#### Parte 2

1. Associare i synset ai termini che pensiamo che massimizzino la similarità (fatto a mano) --> Gia fatto al passo 1 

2. Output costituito da 6 campi: 
   1. Term1
   2. Term2
   3. BS1 (dato da noi)
   4. BS2 (dato da noi)
   5. Sinonimi Term1 (da synset a lemma)
   6. Sinonimi Term2 (da synset a lemma)

   <br/>     
3. **Agreement dell'anotazione Synset trovati:** Tramite il punteggio `kappa di Cohen` calcoliamo il livello 
   di agreement (sklearn) tra i synset trovati da noi

4. **Valutazione annotazione:** Confronto i nostri concetti (synset) con quelli che massimizzano la `cos_sim`
   (Trovati già al passo 3)


### <a href="https://1drv.ms/x/s!AkdbdBkdl_1dhbcgYl1tXDNpjknAIw?e=UYsHF5"> Link all'excel con i dati </a>

### Imports

In [27]:
import hashlib
import matplotlib.pyplot as plt

import re
import csv

## Preparazione

### Organizzazione dei dati

Andiamo a selezionare il dati da utilizzare per l'esercizio 3 tramite la funzione data dal professore

In [1]:
def get_range(surname):
    nof_elements = 500
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % 10)
    idx_intervallo = base_idx * 50+1
    return idx_intervallo
 

input_name = "Bonicco"

values = []
sx = get_range(input_name)
values.append(sx)
dx = sx+50-1
intervallo = "" + str(sx) + "-" + str(dx)
print('{:15}:\tcoppie nell\'intervallo {}'.format(input_name, intervallo))

Bonicco        :	coppie nell'intervallo 51-100


Cambiare il nome dell'annotatore

In [10]:
annotator = "Bonicco"

In [14]:
path_data = "../data/es3_res/it.test.data.txt"
data_file = "SemEval_data_" + annotator + ".tsv"

In [18]:
with open(path_data) as file:
    lines = file.readlines()[50:100]
    lines = [line.rstrip() for line in lines]
    print(lines)

['biotopo\tbiologia', 'magma\tvulcano', 'brainstorming\ttelescopio', 'livello\tpunteggio', 'centesimo\taffare', 'partito politico\tassociazione', 'tsunami\tmare', 'struzzo\tfrutteto', 'cannella\tcaramella', 'scopa\tpolvere', 'galassia\tastronomo', 'succo\tfrappè', 'tapparella\ttenda', 'criminale\tcolpevole', 'cancro al pancreas\tchemioterapia', 'passato\tantecedente', 'Nazioni Unite\tParlamento Europeo', 'canzone\tesecutore', 'pistola\ttaccuino', 'acetilcolina\tiride', 'alfabeto\tpenna', 'coro\tcantante', 'legno\tcoperta', 'soldato\tpace', 'inglese\tamericano', 'lucertola\tcoccodrillo', 'denaro\tcontante', 'Polpo Paul\tpolpo', 'calendario\tvacanza', 'base\tsostanza chimica', 'governo\tlucertola', 'JPY\triciclaggio di denaro', 'DeepMind\tGoogle DeepMind', 'scacchi\tscacco al re', 'sonetto\tbellezza', 'lavoratore\tufficio', 'valuta\tscambio', 'poliestere\tcotone', 'tribunale\tgiustizia', 'coda\tretro', 'incidente\tlibro', 'groppo\tvento', 'piano\tarmonia', 'sultano\tministero', 'deserto\

In [19]:
# open(data_file, 'w').close() # Puliamo il file con i termini di riferimento

In [31]:
terms_sim_score = []

with open(data_file, 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "Similarity"])
    
    for tuple in lines:
        w_couple = re.split(r'\t+', tuple.rstrip('\t'))
        print(w_couple)
        #! Scommentare per annotazione "automatica"
        # score = input("\tInserisci il valore di similarità [0 4] per i * "+ str(w_couple) +" *:")
        
        tsv_writer.writerow(w_couple)


['biotopo', 'biologia']
['magma', 'vulcano']
['brainstorming', 'telescopio']
['livello', 'punteggio']
['centesimo', 'affare']
['partito politico', 'associazione']
['tsunami', 'mare']
['struzzo', 'frutteto']
['cannella', 'caramella']
['scopa', 'polvere']
['galassia', 'astronomo']
['succo', 'frappè']
['tapparella', 'tenda']
['criminale', 'colpevole']
['cancro al pancreas', 'chemioterapia']
['passato', 'antecedente']
['Nazioni Unite', 'Parlamento Europeo']
['canzone', 'esecutore']
['pistola', 'taccuino']
['acetilcolina', 'iride']
['alfabeto', 'penna']
['coro', 'cantante']
['legno', 'coperta']
['soldato', 'pace']
['inglese', 'americano']
['lucertola', 'coccodrillo']
['denaro', 'contante']
['Polpo Paul', 'polpo']
['calendario', 'vacanza']
['base', 'sostanza chimica']
['governo', 'lucertola']
['JPY', 'riciclaggio di denaro']
['DeepMind', 'Google DeepMind']
['scacchi', 'scacco al re']
['sonetto', 'bellezza']
['lavoratore', 'ufficio']
['valuta', 'scambio']
['poliestere', 'cotone']
['tribunale'

## Inizio Esercizio